In [1]:
#----------------------------------------------------------------------------------
#--
#-- Company: CTTC
#-- Author: Jorge Iglesias, 2024. jorgeiglesiascostas(at)gmail.com
#--
#-- File Name: multipleChain_singleDMA_system.ipybn
#-- Target Devices: RFSoC4x2 and ZCU208
#-- Description: Initialize and configure the overlay (ADC, FT and ChS) to implement
#                a front-end which separately transfers four bands of the frequency 
#                spectrum to baseband which will then be sent to the next device via
#                DMA through the "dma2udp" code.
#                This project has been used to send four GNSS bands to a GNSS
#                receiver that performs satellite navigation
#--
#----------------------------------------------------------------------------------
#--
#-- Advanced-RFSoC-front-end is an Advanced multi-band GNSS SDR front-end
#                                  implementation in an RFSoC integrated circuit
#-- This file is part of Advanced-RFSoC-front-end
#--
#-- Copyright (C) 2024 (see AUTHORS file for a list of contributors)
#-- SPDX-License-Identifier: GPL-3.0-or-later
#--
#----------------------------------------------------------------------------------

In [2]:
import xrfdc
from pynq.overlays.base import BaseOverlay
import numpy as np

base = BaseOverlay('base.bit') #Load the overlay

In [3]:
#--- Overlay information ---
#Check if the overlay has been loaded correctly

base?

In [4]:
#--- Overlay initializations ---

base.init_rf_clks() #Initialize the LMK and LMX clocks
base.init_gpio_axi_ctrl() #Initialize the AXI GPIO IP cores

In [5]:
#--- Parameters ---

ADC_sample_frequency = 3600e6 #3.6GHz
RF_RX_frequency_hz=1575.42e6 #DDC mixer frequency

number_samples = 8000

decim_factor = 8 #DDC decimator factor
FabWdValdWords = 8 #DDC parallel samples in AXIS bus
decim_factor_fir = 100 #FIR decimator factor

FabClkOutDivFactor=8
match FabClkOutDivFactor:
    case 1:
         FabClkOutDiv=0x1
    case 2:
         FabClkOutDiv=0x2
    case 4:
         FabClkOutDiv=0x3
    case 8:
         FabClkOutDiv=0x4
    case 16:
         FabClkOutDiv=0x5
    case _:
        print("ERROR: Invalid FabClkOutDivFactor!")
        FabClkOutDiv=0

outputDDC_sample_frequency = ADC_sample_frequency/decim_factor
output_sample_frequency = outputDDC_sample_frequency/decim_factor_fir

#Required AXIS Tile clock and Current FabClk have to have the same value
print(f"Sampling frequency after Tile decimation is: {str(outputDDC_sample_frequency/1e6)} [Msps]")
print(f"Required AXIS Tile clock is {str((ADC_sample_frequency/(FabWdValdWords*decim_factor))/1e6)} [MHz]")
print(f"Current FabClk (AXIS Tile clock) output in Tile is set to {str((ADC_sample_frequency/(FabClkOutDivFactor*8))/1e6)} [MHz]")

Sampling frequency after Tile decimation is: 450.0 [Msps]
Required AXIS Tile clock is 56.25 [MHz]
Current FabClk (AXIS Tile clock) output in Tile is set to 56.25 [MHz]


In [6]:
#--- ADC configurations ---
#It is important to maintain the order in which the ADC is configured

adc = base.radio.receiver.channel_20

adc.adc_block.NyquistZone = 1
adc.adc_block.DecimationFactor = decim_factor
adc.adc_block.FabRdVldWords = FabWdValdWords

adc.adc_tile.DynamicPLLConfig(1, 491.52, ADC_sample_frequency/1e6)

adc.adc_block.MixerSettings = {
 'CoarseMixFreq'  : xrfdc.COARSE_MIX_BYPASS,
         'EventSource'    : xrfdc.EVNT_SRC_TILE,
         'FineMixerScale' : xrfdc.MIXER_SCALE_1P0,
         'Freq'           : -RF_RX_frequency_hz/1e6,
         'MixerMode'      : xrfdc.MIXER_MODE_R2C,
         'MixerType'      : xrfdc.MIXER_TYPE_FINE,
         'PhaseOffset'    : 0.0
     }
adc.adc_block.UpdateEvent(xrfdc.EVENT_MIXER)
adc.adc_tile.SetupFIFO(True)

adc.adc_tile.FabClkOutDiv=FabClkOutDiv 

In [7]:
#--- Frequency Translator configurations ---

floatToFixed = 2**(32-6) #ap_fixed<32,6>
TWO_PI = 2 * np.pi

freq_trans = []
freq_trans.append(adc.freq_translator_0)
freq_trans.append(adc.freq_translator_1)
freq_trans.append(adc.freq_translator_2)
freq_trans.append(adc.freq_translator_3)

#Reset the FT IP core
base.freqtransctrlin[0].off() #ap_start
base.freqtransctrlin[1].off() #ap_continue
base.freqtransctrlin[2].on()  #reset
base.freqtransctrlin[2].off()

#Each complex multiplier frequency to perform the second translation to baseband
subBand_frequency_hz = []
subBand_frequency_hz.append(0.5e6)
subBand_frequency_hz.append(1.5e6)
subBand_frequency_hz.append(2.5e6)
subBand_frequency_hz.append(3.5e6)

#Bypass mode
freq_trans[0].register_map.bypass = 0
freq_trans[1].register_map.bypass = 0
freq_trans[2].register_map.bypass = 0
freq_trans[3].register_map.bypass = 0

#Load parameters to the FTs
for n in range(0, len(freq_trans)):
    phase_inc_single = TWO_PI * (subBand_frequency_hz[n]/output_sample_frequency)

    for i in range(0, FabWdValdWords+1):
        nco_phase_accum = (i * phase_inc_single)
        if nco_phase_accum>TWO_PI:
            while nco_phase_accum>(TWO_PI):
                nco_phase_accum-=TWO_PI;
        elif nco_phase_accum<(-TWO_PI):
            while nco_phase_accum<(-TWO_PI):
                nco_phase_accum+=TWO_PI;
        nco_phase_accum_fixed = np.int32(np.round(nco_phase_accum * floatToFixed))

        if (i <= 0): freq_trans[n].register_map.nco_phase_accum_1 = nco_phase_accum_fixed
        elif (i == 1): freq_trans[n].register_map.nco_phase_accum_2 = nco_phase_accum_fixed
        elif (i == 2): freq_trans[n].register_map.nco_phase_accum_3 = nco_phase_accum_fixed
        elif (i == 3): freq_trans[n].register_map.nco_phase_accum_4 = nco_phase_accum_fixed
        elif (i == 4): freq_trans[n].register_map.nco_phase_accum_5 = nco_phase_accum_fixed
        elif (i == 5): freq_trans[n].register_map.nco_phase_accum_6 = nco_phase_accum_fixed
        elif (i == 6): freq_trans[n].register_map.nco_phase_accum_7 = nco_phase_accum_fixed
        elif (i == 7): freq_trans[n].register_map.nco_phase_accum_8 = nco_phase_accum_fixed
        else: freq_trans[n].register_map.phase_inc_rad = nco_phase_accum_fixed

base.freqtransctrlin[0].on() #ap_start
base.freqtransctrlin[1].on() #ap_continue

In [8]:
#--- Chain Selector configurations ---

chain_sel_re = adc.chain_selector_0
chain_sel_im = adc.chain_selector_1

base.chainselctrlin[0].off() #ap_start
base.chainselctrlin[1].off() #ap_continue

#Chain on (True) or off (False)
chain_sel_re.register_map.s0 = True
chain_sel_im.register_map.s0 = True
chain_sel_re.register_map.s1 = True
chain_sel_im.register_map.s1 = True
chain_sel_re.register_map.s2 = True
chain_sel_im.register_map.s2 = True
chain_sel_re.register_map.s3 = True
chain_sel_im.register_map.s3 = True

base.chainselctrlin[0].on() #ap_start
base.chainselctrlin[1].on() #ap_continue